In [1]:
import os

os.environ["AICORE_AUTH_URL"] = "https://btp-ai-developments-sl2f9ys4.authentication.eu10.hana.ondemand.com"
os.environ["AICORE_CLIENT_ID"] = "sb-38176009-b499-470f-a3b8-9cf98daac1d0!b503699|aicore!b540"
os.environ["AICORE_CLIENT_SECRET"] = "1ac5c77f-d5ac-4e2d-8c19-6ffc47113ec8$52U4q9NYAN-GBm23a2lm_SFVrzmWNhuS7l_qFXs4s4A="
os.environ["AICORE_BASE_URL"] = "https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com"
os.environ["AICORE_RESOURCE_GROUP"]="default"

In [2]:
from llama_index.llms.langchain import LangChainLLM
from gen_ai_hub.proxy.langchain.init_models import init_llm,init_embedding_model

llm = LangChainLLM(llm=init_llm('gpt-4o-mini', max_tokens=4096, temperature = 0))

In [3]:
from llama_index.core import SQLDatabase,Settings

embeddings = init_embedding_model('text-embedding-ada-002')
Settings.llm = llm
Settings.embed_model = embeddings

In [20]:
from urllib.parse import quote_plus
from sqlalchemy import create_engine
from llama_index.core import SQLDatabase

host = 'cfe32093-429a-4e59-87dc-9f3e4da891bf.hna2.prod-eu10.hanacloud.ondemand.com'
port = '443'
user = 'DBADMIN'
password = quote_plus('Bcone@1234567')  # ✅ URL-encode the password

connection_str = f"hana+hdbcli://{user}:{password}@{host}:{port}"

engine = create_engine(connection_str)
sql_database = SQLDatabase(engine, schema='INVTRISK')


In [ ]:
data_context = f"""
        
    """

In [ ]:
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.prompts.base import PromptTemplate, PromptType

table_names = sql_database.get_usable_table_names()

query_engine_sql = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=table_names,
    #context_str_prefix = data_context,
    llm = llm)

In [22]:
ans = query_engine_sql.query("what are the tables in db").response
print(ans)

The database contains the following tables:

1. cost
2. dc_receipts
3. demand_fulfillment
4. location_product
5. location_resource
6. location_source
7. product
8. production_source_header
9. profit_margin
10. resource
11. review_capacity
12. review_component
13. review_dc
14. review_plant
15. review_vendors
16. uom_conversion_factor


In [8]:
ans = query_engine_sql.query("give me all details for product FG-1000 from all available tables")
print(ans)

The SQL query you attempted to execute resulted in an error due to a mismatch in the number of columns across the different SELECT statements used in the UNION ALL operation. Each SELECT statement in a UNION must return the same number of columns, and they must have compatible data types.

To resolve this issue, you need to ensure that each SELECT statement returns the same number of columns. Here’s a suggested approach:

1. **Identify Common Columns**: Determine a common set of columns that can be included in each SELECT statement. For example, you might want to include `product_id` in all queries and add placeholder values (like NULL) for missing columns.

2. **Modify the Query**: Adjust each SELECT statement to return the same number of columns. For instance, if one table returns two columns and another returns one, you can add a NULL or a default value for the missing column in the second query.

Here’s an example of how you might modify the query:

```sql
SELECT product_id, produc

In [10]:
ans = query_engine_sql.query("what is transportation lot size for product RM-2000")
print(ans)

The transportation lot size for product RM-2000 is consistently 2000 units across the locations PL1000 and PL2000. There is no specified maximum transportation lot size for this product.


In [17]:
ans = query_engine_sql.query("please provide me count of overstock situations for product FG-1000 at location PL1000. Refer Review_plant table")
print(ans)

The count of overstock situations for product FG-1000 at location PL1000 is 8.


In [18]:
sql_query = ans.source_nodes[0].node.metadata['sql_query']
sql_query_str = str(sql_query)
print(sql_query_str)

SELECT COUNT(*) AS overstock_count FROM review_plant WHERE product_id = 'FG-1000' AND location_id = 'PL1000' AND total > 0


In [27]:
ans

Response(response='It seems there was an error in the SQL query due to an invalid column name. The query attempted to select details for the product with ID \'FG-1000\', but the column name "Product_ID" may not exist in the database table. Please verify the column names in the "product" table and ensure that "Product_ID" is correctly spelled and exists. Once the correct column name is identified, the query can be adjusted accordingly to retrieve the product details.', source_nodes=[NodeWithScore(node=TextNode(id_='66019f49-abcc-4bfd-bd20-b648ebb07993', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Error: Statement "SELECT Product_ID, Product_Desc, ABC_Code FROM INVTRISK.product WHERE Product_ID = \'FG-1000\'" is invalid SQL.\nError: (260, \'invalid column name: PRODUCT_ID: line 1 col 8 (at pos 7)\')', mimetype='text/plain', start_char_idx=None, end_char_id